In [1]:
import random
import math
import numpy as np
import Parsing_midi as pm
import Init_chromo as init
import Melody_matching as mm
import Cal_score as cs
import count_CE as ce
import mido
from mido import Message, MidiFile, MidiTrack
from collections import Counter
import copy
import matplotlib.pyplot as plt
from itertools import combinations # product: 排列
import time
import multiprocessing as mp
import pretty_midi

/home/hsuan/env1/lib/python3.6/site-packages/pydub/utils.py:165: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


### Seperate phrase
- one bar = 1920 (4 beats)
- sliding window = 7680 (4 bar)

In [2]:
midi_list = pm.get_midi('MidiSample/star.mid', 0)


In [3]:
# tt = pm.get_midi('Output_midi/sky_20.25_136.mid', 0)
# tt

In [4]:
def trans_note_list(song):
    return [song[i].note for i in range(len(song))]

note_msg, midi_msg = pm.separate_song(midi_list, 9600) 
parent = midi_msg[0]  # can change here for testing

def flatten_list(midi_list):
    return [item for sublist in midi_list for item in sublist]

In [5]:
## 4 beats (1920) in one bar
note_msg, midi_msg = pm.separate_song(parent, 1920)
origin_note = [trans_note_list(midi_msg[i]) for i in range(len(midi_msg))]

In [6]:
# ## transform to C major (-3)
# for i in range(len(parent)):
#     parent[i].note = parent[i].note +11 # shift    
# pm.to_midi_file(parent, 8 , 'ori.mid')

### Evolutionary algorithmn
- selection
- crossover
- mutation
- fitness

In [7]:
CE = ce.get_ce_list(midi_msg)  # CE list from every bar

## Selection
def get_fitness_score(new_midi):
    score = 0
    global temp
    # melody matching
    ori = flatten_list(midi_msg)
    new = flatten_list(new_midi)
    smooth_area_score = mm.count_area_score(ori, new, smooth_curve=True)  # smooth
    area_score = mm.count_area_score(ori, new, smooth_curve=False)        # without smooth
    
    for i in range(len(CE)): # 分小節計算分數
        ## encoding
        try:
            temp = copy.deepcopy(new_midi) # 整個midi       
#             note = trans_note_list(temp[i])# 取出一小節  # TODO可能超過原本的小結數
            _score, mid = fitness(temp[i], CE[i], origin_note[i]) # score per bar
            score += _score
        except:
            pass
#     return score
    return score + area_score + smooth_area_score

def select_best(pop_list):  # used in initialization
    pop_idx = [i for i in range(len(pop_list))]
    score = -10000; pop1 = []; pop2 = []
    candidate = list(combinations(pop_idx, 2))
    for i in range(len(candidate)):
#         print("=====================candidate {}-{}=============================".format(len(candidate), i))
        individual1, individual2 = pop_list[candidate[i][0]], pop_list[candidate[i][1]]
        ind_note1, ind_midi1 = pm.separate_song(individual1, 1920) 
        ind_note2, ind_midi2 = pm.separate_song(individual2, 1920) 
        
#         print("individual 1")
        score1 = get_fitness_score(ind_midi1)  # several bars
        
#         print("individual 2")
        score2 = get_fitness_score(ind_midi2)

        total_score = score1 + score2
        if score < total_score :
            score = total_score
            pop1, pop2 = ind_midi1, ind_midi2
        
    return pop1, pop2, score

## Crossover
def crossover(parent, pop):
    ind1 = copy.deepcopy(parent)
    ind2 = copy.deepcopy(pop)   
    if len(ind1)<len(ind2):
        ind2 = ind2[:len(ind1)]
    else:
        ind1 = ind1[:len(ind2)]
        
    crossover_point = random.randint(0,len(ind1)-2)
       
    while crossover_point % 2 != 0:
        crossover_point = random.randrange(2,len(temp)-2, 2)
    if ind2[crossover_point].note != ind2[crossover_point-1].note and \
    ind1[crossover_point].note != ind1[crossover_point-1].note and \
    ind2[crossover_point].note == ind2[crossover_point+1].note and \
    ind1[crossover_point].note == ind1[crossover_point+1].note:
        pop_midi1 = ind1[ :crossover_point] + ind2[crossover_point: ]
        pop_midi2 = ind2[ :crossover_point] + ind1[crossover_point: ]    
    else:
        pop_midi1 = ind1
        pop_midi2 = ind2
        
    return pop_midi1, pop_midi2
    
## Mutation
def mutation(child): 
    temp = copy.deepcopy(child)
    mute_point = random.randint(0,len(temp)-1)  # choose the mutation point
    while mute_point % 2 != 0:
        mute_point = random.randint(0,len(temp)-1)
    mutate_note = init.get_note(temp[mute_point].note)
    if temp[mute_point].note + mutate_note > 50 and temp[mute_point].note + mutate_note < 90:
        temp[mute_point].note = temp[mute_point].note + mutate_note
        temp[mute_point + 1].note = temp[mute_point].note
    return temp


## Fitness
C_pitch = [0, -1, 2, -1, 4, 5, -1, 7, -1, 9, -1, 11]  # there is no sharp and flat in C scale, so set it as -1
chord_list = [[0,4,7],[-1,-1,-1],[2,5,9],[-1,-1,-1],[4,7,11],[5,9,0],[-1,-1,-1],[7,11,2],[-1,-1,-1],[9,0,4],[-1,-1,-1],[11,2,5]]
pitch_list = ['C','D','E','F','G','A','B']

harmony = [[0,2,4,7,-3,-5,-1], [0], [0,5,-2,-3,-7,2,3],[0], [0,3,1,-4,-2,-7], [0,2,4,-1,-3,-6],\
[0], [0,2,4,7,-2,-3,-5], [0],[0,2,3,7,-2,-7,-5],[0],[1,3,0,-4,-2,-9]]

invalid = [[],[0],[],[0],[],[],[0],[],[0],[],[0],[]]


def fitness(midi_msg, chord, ori_note): # input type: list
    score_ce = 0; score_harmony = 0
    scale_list = [0,2,4,5,7,9,11]  # = midi notes = ['C','D','E','F','G','A','B']
    
    score_h = cs.harmony_score(midi_msg, chord, ori_note)
    
    # rate the CE distance in each bar (30%)
    xyz_ = ce.trans_to_coordinate(midi_msg)
    CE_ = ce.count_CE(xyz_)  
    dist = ce.count_distance(chord, CE_)  # compare new and old CE
    if dist == 0.0:  # the chord remain
        score_ce = 40
    elif dist > 0 and dist <= 2:
        score_ce = -2*(dist)
    elif dist > 3 and dist <= 4:
        score_ce = -5*(dist)
    elif dist > 4 and dist <= 8:
        score_ce = -8*(dist)
    else:
        score_ce = -30     
          
    return round((score_ce + score_h), 2) , midi_msg  # 4 bar

#### [CE Reference paper](https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=7969627) 
#### [Fitness function Paper reference](https://www.jstor.org/stable/pdf/3681713.pdf?refreqid=excelsior%3A161dc33b5d8b13a0a891837b552564c5)

### Main - Multiprocessing
- initialize: 17 combinations
- choose 2 highest score
- do crossover, mutation then generate 60 candicates
- choose 2 highest andidates become next populations

In [9]:
MUTATION_RATE = 0.5
CROSSOVER_RATE = 0.6
GENERATIONS = 200

## Initialize 

pop_list = []
for i in range(200):
    temp = []
    addmelody = init.add_melody(flatten_list(midi_msg))
    addnote = init.add_note(addmelody)
    temp.append(init.change_duration(addnote, 480, 119))
    temp = flatten_list(temp)
    pop_list.append(temp)

NameError: name 'duraiton' is not defined

In [19]:
%%time

# Multiprocess crossover, mutation * N times
def job(candidateq, parent, pop): 
    candidate_list = []; count = 0
    candidate_list.append(parent)
    candidate_list.append(pop)
    for i in range(GENERATIONS):   ##change here        
        # crossover
        crossover_rate = round(random.uniform(0, 1),1)
        if crossover_rate < CROSSOVER_RATE:
            crossover_c1, crossover_c2 = crossover(parent, pop)
        else:
            crossover_c1 = parent 
            crossover_c2 = pop
        # mutation
        mute_rate = round(random.uniform(0, 1),1)    
        if mute_rate < MUTATION_RATE: 
            mutation_c1 = mutation(crossover_c1)
            mutation_c2 = mutation(crossover_c2)
            mutation_c1 = init.add_melody(mutation_c1)
            mutation_c2 = init.add_melody(mutation_c2)
#             mutation_c1 = init.add_note(mutation_c1)
#             mutation_c2 = init.add_note(mutation_c2)
        else:
            mutation_c1 = init.add_melody(crossover_c1)
            mutation_c2 = init.add_melody(crossover_c2)
#             mutation_c1 = (crossover_c1)
#             mutation_c2 = (crossover_c2)
                  
        candidate_list.append(mutation_c1)
        candidate_list.append(mutation_c2)
    candidateq.put(candidate_list)

## get score from all candidates
def job2(scoreq,new_midi,a,b):
    t = []
    for i in range(a,b):
        score = 0
        ind_note1, ind_midi1 = pm.separate_song(new_midi[i], 1920)  
        score = get_fitness_score(ind_midi1)
        t.append([score, ind_midi1])
    scoreq.put(t)
    
def main():
    count = 0; score_list = []
    # choose 2 best candidate from initialization
    c1, c2, scr1 = select_best(pop_list)
    parent = flatten_list(c1)
    pop = flatten_list(c2)
    score_list.append(scr1)
    
    score_list = []
    while count < GENERATIONS:
#         print("------------Generation {}-------------".format(count))
        candidateq = mp.Queue()
        scoreq = mp.Queue()
    
        # generate condidates
        p1 = mp.Process(target = job, args = (candidateq, parent, pop))
        p1.start()
        candidate = candidateq.get()
        p1.join()
        
        # calculate score
        p2 = mp.Process(target = job2, args = (scoreq, candidate, 0, len(candidate))) # 200 candidate
        p2.start()
        s = scoreq.get()
        p2.join()
        
        score = []; pop = [];  max_score = 0
        for i in range(len(s)):
            score.append(round(s[i][0],3)) # get score list
        max_idx = score.index(max(score))
        max_score = max(score)
        score_list.append(max_score) # only append highest score
        pop.append(s[max_idx]) # 1 candidate
        del s[max_idx]

        max_idx = score.index(max(score))
        pop.append(s[max_idx]) # 2 candidate

        parent = flatten_list(pop[0][1])
        pop = flatten_list(pop[1][1])
        
        # output midi
        pm.to_midi_file(parent, 1 , 'Output_midi/star_{}_{}.mid'.format(str(max_score), str(count)))

        print("Generation", count, "\t: score", max_score)
        count += 1  
    
    # plot score
    plt.plot(score_list)
    plt.xlabel('Generation')
    plt.ylabel('Score')
    plt.show()
          
if __name__ == '__main__':
    main()   

Generation 0 	: score 306.87
Generation 1 	: score 344.84
Generation 2 	: score 344.84
Generation 3 	: score 344.84
Generation 4 	: score 344.84
Generation 5 	: score 344.84
Generation 6 	: score 344.84
Generation 7 	: score 344.84
Generation 8 	: score 344.84
Generation 9 	: score 344.84
Generation 10 	: score 344.84
Generation 11 	: score 344.84
Generation 12 	: score 344.84
Generation 13 	: score 344.84
Generation 14 	: score 344.84
Generation 15 	: score 344.84
Generation 16 	: score 344.84
Generation 17 	: score 344.84
Generation 18 	: score 344.84
Generation 19 	: score 344.84
Generation 20 	: score 344.84
Generation 21 	: score 344.84
Generation 22 	: score 344.84
Generation 23 	: score 344.84
Generation 24 	: score 344.84
Generation 25 	: score 344.84
Generation 26 	: score 344.84
Generation 27 	: score 344.84
Generation 28 	: score 344.84
Generation 29 	: score 344.84
Generation 30 	: score 344.84
Generation 31 	: score 344.84
Generation 32 	: score 344.84
Generation 33 	: sco

KeyboardInterrupt: 

In [ ]:
melody_series = [[0,-5,0,4],[0,0,0,0], [0,-8,-3,0],[0,0,0,0],[0,-9,-4,0],[0,-8,-3,0],[0,0,0,0], [0,-7,-3,0],[0,0,0,0], [0,-7,-4,0],[0,0,0,0],[0,-9,-4,0]]
melody_series2 = [[0,-1,0,2],[0,0,0,0], [0,-5,-3,0],[0,0,0,0],[0,-4,-2,0],[0,-1,-3,0],[0,0,0,0], [0,-7,-5,0],[0,0,0,0], [0,-2,0,2],[0,0,0,0],[0,-2,-4,0]]
melody_list = [melody_series, melody_series2]

melody_idx = int(np.random.choice(2, 1, replace=False, p=[0.5,0.5]))
interval = melody_list[melody_idx]


In [ ]:
# 將note依照time拆成多個
def to_note_freq(midi_msg):     
    note_freq = []; count = 0
    for i in range(len(midi_msg)):
        if i % 2 == 0:
            for j in range(int((midi_msg[i].time + midi_msg[i+1].time)/240)):
                count += 1
                note_freq.append([midi_msg[i].note, count])
    return note_freq

# 計算面積差  
def count_area(ori_y, new_y):  
    area = 0
    for i in range(len(ori_y)):
        area += abs(ori_y[i]-new_y[i])
    return area

# smooth melody curve, sliding window = 7
def smooth(note_seq):  
    temp = []
    for i in range(len(note_seq)):
        if i < 4:
            temp.append(note_seq[i])
        elif i > len(note_seq) - 4:
            temp.append(note_seq[i])
        else:
            median = int(statistics.median(note_seq[i-4:i+4]))
            temp.append(median)
    return temp


# 校正頭尾
def correct_xy(xy1, xy2):
    if len(xy1) > len(xy2):
        xy1 = xy1[0:len(xy2)]
    else:
        xy2 = xy2[0:len(xy1)]
    return xy1, xy2

# melody matching
def count_area_score(ori_midi, new_midi, smooth_curve=True):  # smooth
    score = 0
    a = to_note_freq(ori_midi)
    b = to_note_freq(new_midi)
    a, b = correct_xy(a,b)
    # 拆成XY座標
    x = [i[1] for i in a]
    y = [i[0] for i in a]

    x2 = [i[1] for i in b]
    y2 = [i[0] for i in b]
    
    # smooth
    y_s1 = smooth(y)
    y_s2 = smooth(y2)

    if smooth_curve:
        area_diff = count_area(y_s1, y_s2)
#         print("smooth",area_diff)
        if area_diff > 0 and area_diff <= 90:
            score -= (area_diff / 10 * 2)
        if area_diff > 0 and area_diff <= 65:
            score += 15
        elif area_diff > 90:
            score -= 30 
    else:  
        area_diff = count_area(y, y2)
#         print("not smooth", area_diff)
        if area_diff > 0 and area_diff <= 70:
            score += (area_diff / 10 * 3)

        else:
            score += 0 
    return a,b

In [ ]:
def parson(song):
    n_list = []; parson = []
    for i in range(len(song)):
        if i % 2 == 0:
            n_list.append(ori[i].note)

    for i in range(len(n_list)-1):
        if n_list[i] > n_list[i+1]:
            parson.append(-1)
        elif n_list[i] < n_list[i+1]:
            parson.append(+1)
        else:
            parson.append(0)

    temp=[]; t=0
    for i in range(len(parson)-1):
        t += parson[i]
        temp.append(t)
    return temp

In [ ]:
parson(new)

In [ ]:
score = 0
a = to_note_freq(midi_msg[2])
# b = to_note_freq(new_midi)
# a, b = correct_xy(a,b)
# 拆成XY座標
x = [i[1] for i in a]
y = [i[0] for i in a]

# x2 = [i[1] for i in b]
# y2 = [i[0] for i in b]

# smooth
y_s1 = smooth(y)
# y_s2 = smooth(y2)


In [ ]:
import matplotlib.pyplot as plt
plt.plot(x, y, color='green', linewidth=2, label='original')
plt.plot(x, y_s1, 'r--', linewidth=2, label='after smooth')
plt.xlabel('midi time')
plt.ylabel('midi note')
plt.legend(loc='upper left')
plt.savefig('result.png')